# Ajuste de Curvas (Fitting) usando a SciPy

Uma das tarefas comuns mais importantes que um físico (e até mesmo químicos) tem que fazer é a de ajustar os dados experimentais em um modelo teórico. Neste Notebook, no primeiro exemplo mostra-se como fazer o ajuste de dados em um modelo teórico e o segundo exemplo ilustra uma das armadilhas comuns na montagem de curva (não convergente para um ajuste aceitável) e como remediar a situação.

Não iremos lidar com a leitura de um arquivo de dados neste Notebook (apenas para que possamos nos concentrar no ajuste da curva), serão gerados alguns dados falsos, adicionando ruído aleatório a uma função analítica. Esta função é, na verdade, nossa função modelo para ajustar nossos dados. Quando se ajusta os dados de um experimento real, é necessário usar um função personalizada  apropriada ao modelo teórico que descreve o experimento e a função `curve_fit` da SciPy irá usar essa função para ajustar seus dados.

Então, aqui estão as bibliotecas necessárias e nossa função de modelagem (que também será usada para gerar dados falsos):

[//]: \begin{equation}
i \hbar \frac{\partial \psi}{\partial t} = H \psi
\end{equation}



In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def fitFunc(t, a, b, c):
    return a*np.exp(-b*t) + c


Agora criamos

- (1) uma matriz numpy (t) com 50 pontos espaçados linearmente entre 0 e 4,
- (2) uma matriz numpy (temp) com 50 temperaturas usando a = 5.0, b = 1.5 e c = 0.5
- (3) uma matriz numpy (ruídosa) com ruído gaussiano adicionado à  matriz da temperatura e, então, usamos a função da SciPy `curve_fit` para encontrar os melhores parâmetros de ajuste da função fitFunc; Os retornos da função `curve_fit` são a matriz de coeficientes de ajuste e uma matriz de covariâncias de ajuste (as diagonais dos quais são os quadrados das incertezas de 1 sigma nos coeficientes de ajuste):

In [ ]:
t0 = 0
tf = 4
N  = 51  # Numero de pontos
t = np.linspace(t0,tf,N)
temp = fitFunc(t, 5.0, 1.5, 0.5) # função
ruido = temp + 0.25*np.random.normal(size=len(temp))
fitParams, fitCovariances = curve_fit(fitFunc, t, ruido)
print ('Coeficientes do ajuste:\n', fitParams)
print ('Matriz de Covariância:\n', fitCovariances)

Aqui está um gráfico dos dados (falso) e o melhor ajuste:

In [ ]:
largura = 10  # Largura da figura
altura  = 6   # Altura da figura
plt.figure(figsize =(largura, altura))     # Define o tamanho do gráfico

plt.ylabel('Temperatura (C)', fontsize = 16)
plt.xlabel('tempo (s)', fontsize = 16)
plt.xlim(0,4.1)
# Faz o gráfico dos dados como círculos vermelhos e com barras de erro verticais
plt.errorbar(t, ruido, fmt = 'ro', yerr = 0.2)
# agora o gráfico da curva de melhor ajuste e também as curvas sigma +- 1 
# (A raiz quadrada do elemento da diagonal da matriz de covariância é a 
#  incerteza no parâmetro de ajuste.)
sigma = [np.sqrt(fitCovariances[0,0]), \
         np.sqrt(fitCovariances[1,1]), \
         np.sqrt(fitCovariances[2,2]) ]
plt.plot(t, fitFunc(t, fitParams[0], fitParams[1], fitParams[2]),\
         t, fitFunc(t, fitParams[0] + sigma[0], fitParams[1] - sigma[1], fitParams[2] + sigma[2]),\
         t, fitFunc(t, fitParams[0] - sigma[0], fitParams[1] + sigma[1], fitParams[2] - sigma[2])\
         )
plt.show()
# Salvar um gráfico em um arquivo
plt.savefig('dataFitted.pdf', bbox_inches=0, dpi=600)

# Outro exemplo com hipótese iniciais para parâmetros de ajuste

A função `curve_fit` da SciPy usa valores de 1.0 como *chutes iniciais* para os parâmetros de ajuste. Isso funcionou bem no problema anterior, mas, às vezes, é confrontado com uma situação em que o ajuste não converge, e neste caso, você (como o usuário inteligente) deve fornecer uma estimativa inteligente para um ponto de partida.

Aqui eu defino alguns dados igualmente espaçados no tempo, e uma série de termperaturas que esperamos encaixar em um exponencial que remanesce de um capacitor de carga. Eu levo essa variedade de temperaturas e adiciono algum ruído aleatório e também define barras de erro nos valores de duração: 


In [ ]:
def cap(x, a, b):
    return a*(1.0-np.exp(b*x))
t = np.linspace(0.4,2.0, 9)
#temps = cap(t,25,-2)
temps = np.array([13.76, 17.47, 19.95, 21.62, 22.73, 23.48, 23.98, 24.32, 24.54])
temps = temps + 0.4*np.random.normal(size=len(temps))
dTemps = np.array([1.2, 0.9, 1.0, 0.8, 0.7, 0.7, 0.6, 0.5, 0.5])

In [ ]:
# Se você executar este código, não funcionará. Olhe a mensagem de erro resultante:
fitParams, fitCovariances = curve_fit(cap, t, temps)
print (fitParams)
print (fitCovariances)

Agora eu adiciono algumas hipóteses iniciais para a e b; Observe o ajuste agora funciona bem (pense em p0 como uma matriz de valores de parâmetro em t = 0):

In [ ]:
fitParams, fitCovariances = curve_fit(cap, t, temps, p0 = [25.0, -1.0])
print ('Coeficientes do ajuste:\n', fitParams)
print ('Matriz de Covariância:\n', fitCovariances)

In [ ]:
largura = 10  # Largura da figura
altura  = 6  # Altura da figura
plt.figure(figsize =(largura, altura))     # Define o tamanho do gráfico
#
plt.ylabel('Temperatura (C)', fontsize = 16)
plt.xlabel('tempo (s)', fontsize = 16)
plt.xlim(0,2.2)
plt.ylim(12, 26)
plt.errorbar(t, temps, yerr = dTemps, xerr = 0.1, fmt='ro')
plt.plot(t, cap(t, fitParams[0], fitParams[1]),
     t, cap(t, fitParams[0] + np.sqrt(fitCovariances[0,0]), fitParams[1] - np.sqrt(fitCovariances[1,1])),
     t, cap(t, fitParams[0] - np.sqrt(fitCovariances[0,0]), fitParams[1] + np.sqrt(fitCovariances[1,1]))
     )
plt.show()

# Numpy Ajuste de Cuvas com Incertezas de $1\sigma$ e $3\sigma$ 

Agora será criado um ruído em cima de uma decomposição exponencial (que é tratada como dados falsos), usa a função `curve_fit` da Scipy para encontrar os melhores valores de ajuste e incertezas associadas. Então, cria-se todos os 8 únicos e possíveis ajustes permitidos pelas 3 parâmetros $\pm$ incertezas, calcula-se o desvio padrão de seus valores em cada tempo (eixo dos x) e usa-se isso para fazer o gráfico barras de erro de $1\sigma$ e $3\sigma$ das curvas para ajudar o usuário a visualizar a incerteza sobre o ajuste.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def func(x, a, b, c):
    return a*np.exp(-b*x) + c

# O usuário fornece o código LaTeX para a função para uso posterior:
fitEquation = r"$\displaystyle\mathrm{fit}(t) =  a e^{-b t} + c$"

N=100
x = np.linspace(0,4, N)       ## foi usado arbitrariamente N = 100 pontos
dx = (max(x) - min(x))/N
y = func(x, 2.5, 1.3, 0.5)    # função base
yn = y + 0.2*np.random.normal(size=len(x))  # adicionando o ruído 
# ajuste da curva
popt, pcov = curve_fit(func, x, yn)  # popt = Parâmetros ótimos para o ajuste; Matriz de Covariância

# sqrt(elementos diagonais ) de pcov são desvios de 1 sigma 
sigma = np.sqrt([pcov[0,0], pcov[1,1], pcov[2,2]])

print ('Coeficientes do ajuste:\n', popt)
print ('Matriz de Covariância: \n', pcov)


A próxima célula é grosseira, e eu só queria uma resposta.

(TODO :) Sem dúvida, eu poderia generalizar este código para que ele olhasse a quantidade de parâmetros na função de ajuste fornecida pelo usuário, e então resolver todas as possibilidades combinatórias para curvas de ajuste.

In [ ]:
valores = np.array([
    func(x, popt[0] + sigma[0], popt[1] + sigma[1], popt[2] + sigma[2]), 
    func(x, popt[0] + sigma[0], popt[1] - sigma[1], popt[2] + sigma[2]),   
    func(x, popt[0] + sigma[0], popt[1] + sigma[1], popt[2] - sigma[2]), 
    func(x, popt[0] + sigma[0], popt[1] - sigma[1], popt[2] - sigma[2]), 
    func(x, popt[0] - sigma[0], popt[1] + sigma[1], popt[2] + sigma[2]), 
    func(x, popt[0] - sigma[0], popt[1] - sigma[1], popt[2] + sigma[2]),
    func(x, popt[0] - sigma[0], popt[1] + sigma[1], popt[2] - sigma[2]), 
    func(x, popt[0] - sigma[0], popt[1] - sigma[1], popt[2] - sigma[2]) 
    ])

In [ ]:
#print (valores)
print (valores[ : ,0])

A seguir o erro de ajuste representa o desvio padrão de todos os possíveis ajustes $\pm$ incerteza dos valores em cada posição x. Pode-se imaginar obter os desvios mínimos e máximos possíveis, mas este é um comando de uma linha que é bastante simples e rápido.

In [ ]:
#print (fitError)
fitError = np.std(valores, axis=0)

In [ ]:
from matplotlib.ticker import MaxNLocator

nSigma = 3
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
plt.rcParams['xtick.major.pad'] = 10
plt.rcParams['ytick.major.pad'] = 10

fig = plt.figure(figsize=(12,8), dpi=100, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax.xaxis.labelpad = 20
ax.yaxis.labelpad = 20
curveFit = func(x,popt[0], popt[1], popt[2] )
plt.plot(x, yn, 'o')
#plt.hold(True)
plt.plot(x, curveFit, 
    linewidth=2.5, 
    color = 'green',
    alpha = 0.6,
    label = fitEquation)

plt.bar(x, 
    height = 2*nSigma*fitError,  
    width=dx, 
    bottom = curveFit - nSigma*fitError, 
    orientation = 'vertical', 
    alpha=0.2, 
    color = 'purple',
    edgecolor = None,
    label = r"$\mathrm{Barras\;\;de\;\;erros\;\;de\;}\displaystyle \pm 3\sigma$")

plt.plot(x, curveFit+fitError, 
    linewidth = 1.0, 
    alpha = 0.8, 
    color = 'red',
    label = r"Erro de $\pm 1\sigma$")

plt.plot(x, curveFit-fitError, 
    linewidth = 1.0, 
    alpha = 0.8, 
    color = 'red')
props = dict(boxstyle='round', facecolor='wheat', alpha=0.2)
plt.text(2.8, 1.2, 
    ("$\mathrm{Valores\;\;da\;\;curva\;\;ajustada:\;}$\n a = %.3f\t $\pm$ %.3f\n b = %.3f\t $\pm$ %.3f \n c = %.3f\t $\pm$ %.3f" 
    % (popt[0], sigma[0], popt[1], sigma[1], popt[2], sigma[2])), fontsize=16,bbox=props )
plt.xlabel(r'\textrm{tempo (s)}', fontsize=24)
plt.ylabel(r'\textrm{temperatura (K)}',fontsize=24)
plt.title(r"Ajuste exponencial com erros de $\pm 1\sigma$ e $\pm 3\sigma$",
      fontsize=28, color='k')
ax.legend(fontsize=18)
plt.gca().xaxis.set_major_locator(MaxNLocator(prune='lower'))
plt.savefig('3sigmaPlot.pdf', figsize=(6,4), dpi=600)
plt.show()

## Exemplo

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

plt.figure(figsize=(12,8))
plt.imshow( np.random.random((100,100)))
plt.gca().xaxis.set_major_locator( MaxNLocator(nbins = 7, prune = 'lower') )
plt.gca().yaxis.set_major_locator( MaxNLocator(nbins = 6) )
cbar = plt.colorbar()
cbar.locator = MaxNLocator( nbins = 6)
plt.show()

# Leitura e Escrita de Arquivos de dados

Aqui está um programa que

1. grava algum texto em um arquivo com o nome `texto.txt`,

2. e depois lê o texto novamente e

3. imprime-o na tela.


<!-- -->


Os dados armazenados no arquivo `texto.txt` são:

```
Escrevendo um texto para o arquivo. Essa é a primeira linha do arquivo.
Já essa é a segunda linha do arquivo.
```

Em mais detalhes, você abriu um arquivo com o comando `open` e atribuiu esse objeto de arquivo aberto à variável `out_file`. Em seguida, escrevemos dados para o arquivo usando o método `out_file.write`. Observe que no exemplo acima, damos uma string ao método `write`. Podemos, é claro, usar toda a formatação que discutimos anteriormente - veja [Impressão formatada](#IO-imp-form) e [Novo estilo de formatar string](#IO-form-str-new). Por exemplo, para gravar este arquivo com a tabela de nomes `tabela.txt`, podemos usar este programa Python. É uma boa prática fechar `close()` os arquivos quando concluímos a sua leitura e a escrita. Se um programa Python for deixado de forma controlada (ou seja, não através de um corte de energia ou de um erro improvável no idioma Python ou no sistema operacional), ele fechará todos os arquivos abertos assim que os objetos do arquivo forem destruídos. No entanto, fechá-los ativamente logo que possível é melhor estilo e o mais seguro.



### Exemplo de leitura de arquivos <a name="IO-exemp-ler-arq"></a>


Utilizamos um arquivo chamado `meu-arq.txt` contendo as seguintes 3 linhas de texto para os exemplos abaixo:


   > Esta é a primeira linha.
     Esta é a segunda linha.
     Esta é a terceira e última linha.
     
Vamos abrir ele com o comando `open()`, o qual retorna um objeto de arquivo, e é comumente usado com dois argumentos: `open(nome-do-arquivo, mode)`.

O primeiro argumento é uma string contendo o nome do arquivo. O segundo argumento é uma outra string contendo alguns caracteres os quais descrevem a maneira que o arquivo será usado. O modo pode ser `'r'` quando o arquivo só será lido, `'w'` apenas para escrever (um arquivo existente com o mesmo nome será apagado) e `'a'` abre o arquivo para acrescentar; quaisquer dados gravados ao arquivo são automaticamente adicionados ao final. `'r+'` abre o arquivo para leitura e escrita. O argumento mode é opcional; `'r'` será assumido se for omitido.

Normalmente, os arquivos são abertos no modo de texto, ou seja, você lê e escreve strings de e para o arquivo, que são codificados em uma codificação específica. Se a codificação não for especificada, o padrão é dependente da plataforma (consulte o manual do comando `open()`). A opção `'b'` anexado ao modo abre o arquivo no modo binário: agora os dados são lidos e escritos sob a forma de objetos bytes. Este modo deve ser usado para todos os arquivos que não contêm texto.

No modo de texto, o padrão quando se lê é converter terminais de linha específicos da plataforma (`\n` no Unix, `\r\n` no Windows) para apenas `\n`. Ao escrever no modo de texto, o padrão é converter ocorrências de `\n` de volta para terminações de linha específicas da plataforma. Esta modificação nos bastidores dos dados de arquivos é ótima para arquivos de texto, mas corromperá dados binários como os arquivos JPEG ou EXE. Tenha muito cuidado para usar o modo binário ao ler e gravar esses arquivos.

In [1]:
f = open('Meu_Arquivo.txt', 'w')
f.write('Esta é a primeira linha.\n'
        'Esta é a segunda linha.\n'
        'Esta é a terceira e última linha.')
f.close()

In [2]:
# Para verificar o conteudo do arquivo
!cat Meu_Arquivo.txt

Esta é a primeira linha.
Esta é a segunda linha.
Esta é a terceira e última linha.

#### fileobject.read()

O método `fileobject.read()` lê todo o arquivo e o retorna como uma string (incluindo novos caracteres de linha).

In [3]:
f = open('Meu_Arquivo.txt', 'r')
texto=f.read()
texto1= texto.split("\n")
print(texto1[:])

['Esta é a primeira linha.', 'Esta é a segunda linha.', 'Esta é a terceira e última linha.']


In [4]:
# Para fechar o arquivo
f.close()

#### fileobject.readlines()

O método `fileobject.readlines()` retorna uma lista de strings, onde cada elemento da lista corresponde a uma linha na string:


In [5]:
f = open('Meu_Arquivo.txt', 'r')
f.readlines()

['Esta é a primeira linha.\n',
 'Esta é a segunda linha.\n',
 'Esta é a terceira e última linha.']

In [6]:
# Para fechar o arquivo
f.close()

Isso é freqüentemente usado para iterar sobre as linhas e fazer algo com cada linha. Por exemplo:

In [7]:
f = open('Meu_Arquivo.txt', 'r')
for line in f.readlines():
    print("%d characters" % len(line))
f.close()

25 characters
24 characters
33 characters


Vamos tratar dados agora, para isso vamos começar criando um subdiretório no sistema

In [ ]:
# Esse comando cria um subdiretório, e ele é específico do linux
! mkdir -p Dados

Na linha seguinte vou escrever os meus dados em um arquivo `Dados/dados0.dat` no subdiretório `Dados`

In [8]:
%%writefile dados0.dat
# x   y    erro_x erro_y
  1   3     0.5    0.5
  4   19    0.5    0.5
  6   28    0.5    1.5
  12  44    0.5    2.5
  20  83    0.5    3.5

Overwriting dados0.dat


In [11]:
# Verificando o conteúdo do arquivo. Comando específico do linux
!cat  dados0.dat

# x   y    erro_x erro_y
  1   3     0.5    0.5
  4   19    0.5    0.5
  6   28    0.5    1.5
  12  44    0.5    2.5
  20  83    0.5    3.5


Agora vamos ler e fazer um gráfico do arquivo

In [14]:
! ls *.dat

dados0.dat


In [12]:
f = open('dados0.dat', 'r')
head0 = f.readline()
#linhas = f.readlines()
print(head0)
linha=[]
for i in range(5):
    linha = f.readline()
    #print(linha)
    c = " ".join(linha.split())
    #print(type(c[0]))
    l = c.split()
    x = int(l[0])
    y = int(l[1])
    xe = float(l[2])
    ye = float(l[3])
    print(c," => ", x, y, xe, ye)
f.close()

# x   y    erro_x erro_y

1 3 0.5 0.5  =>  1 3 0.5 0.5
4 19 0.5 0.5  =>  4 19 0.5 0.5
6 28 0.5 1.5  =>  6 28 0.5 1.5
12 44 0.5 2.5  =>  12 44 0.5 2.5
20 83 0.5 3.5  =>  20 83 0.5 3.5


Uma segunda forma é 

In [13]:
f = open('dados0.dat', 'r')
head0 = f.readline()
#linhas = f.readlines()
print(head0)
linha=[]
for i in range(5):
    linha = f.readline()
    c = linha.lstrip()
    l = c.split()
    print(l)
f.close()

# x   y    erro_x erro_y

['1', '3', '0.5', '0.5']
['4', '19', '0.5', '0.5']
['6', '28', '0.5', '1.5']
['12', '44', '0.5', '2.5']
['20', '83', '0.5', '3.5']


Observe que isso irá ler o arquivo completo em uma lista de strings quando o método `readlines()` é chamado. Isso não é problema se soubermos que o arquivo é pequeno e cabe na memória da máquina.

Se assim for, também podemos fechar o arquivo antes de processarmos os dados, por exemplo:

In [ ]:
f = open('Meu_Arquivo.txt', 'r')
lines = f.readlines()
f.close()
for line in lines:
    print("%d characters" % len(line))

#### Iterando sobre linhas (objeto de arquivo)

Existe uma possibilidade mais nítida de ler um arquivo linha a linha, o qual (i) só lerá uma linha de cada vez (e é, portanto, adequado para arquivos grandes também) e (ii) resulta em código mais compacto:


In [ ]:
f = open('Meu_Arquivo.txt', 'r')
for line in f:
    print("%d characters" % len(line))
    print("%s" % line)
f.close()

In [15]:
f=open('dados0.dat','r')
line = f.readline()  # Lê a primeira linha com o cabeçalho
print(line)
for line in f:
    line = line.rstrip() # remove "\n"
    c = " ".join(line.split()) # Remove espaços em brancos extras
    (x, y, xe, ye) = c.split(' ') # Separa as colunas da linha
    print('x =(',x,'+/-',xe,')  e  y = (',y,'+/-',ye,')')
f.close()

# x   y    erro_x erro_y

x =( 1 +/- 0.5 )  e  y = ( 3 +/- 0.5 )
x =( 4 +/- 0.5 )  e  y = ( 19 +/- 0.5 )
x =( 6 +/- 0.5 )  e  y = ( 28 +/- 1.5 )
x =( 12 +/- 0.5 )  e  y = ( 44 +/- 2.5 )
x =( 20 +/- 0.5 )  e  y = ( 83 +/- 3.5 )


Aqui, o manipulador de arquivos `f` atua como no iterador e retornará a próxima linha em cada iteração subsequente do loop for até o final do arquivo ser alcançado (e então o loop for é terminado).


##### Mais leitura

[Methods of File objects, Tutorial, Section 7.2.1](http://docs.python.org/tutorial/inputoutput.html#methods-of-file-objects)

## Códigos Para Estrutura e Análise de Dados

Vamos gravar os pontos de uma função em um arquivo e depois vamos ler o arquivo e fazer o gráfico dele:

In [ ]:
# permite que se mostre os gráficos nesse ambiente
%matplotlib inline 

In [ ]:
# Fazendo o gráfico da função
import numpy as np                  # obtém acesso aos rápidos arrays da numpy 
import matplotlib.pyplot as plt     # Para fazer os gráficos das funções

# Definindo a função
def f(x):
    return np.exp(x) - 6*x

x0 = -5     # Valor inicial de x
xf = 5      # Valor final de x
dx = 0.1    # Passo da discretização dx = (xf - x0)/(np-1)

x = np.arange(x0, xf, dx)           # cria um vetor de dados x-data
y = f(x)                            # calcula o vetor de dados y-data
y1 = np.exp(x)

n=len(x)

print(n)

In [ ]:
# Cria um arquivo de dados sem formatação
f = open('Dados1.dat', 'w')
for i in range(n):
    f.write(str(x[i])+'\t'+str(y[i])+'\t'+str(y1[i])+'\n')
f.close()

In [ ]:
# Mostra o conteúdo das primeiras linhas do arquivo
!head Dados1.dat

In [ ]:
# Mostra o conteúdo das últimas linhas do arquivo
!tail Dados1.dat

In [ ]:
# conta o número de linhas do arquivo
!wc -l Dados1.dat

In [ ]:
# Cria um arquivo de dados com formatação
f = open('Dados2.dat', 'w')
for i in range(n):
    a = "{0:5.2f}\t\t{1:8.4f}\t\t{2:0<14.9g}\n".format(x[i],y[i],y1[i])
    f.write(a)
f.close()

In [ ]:
!head Dados2.dat

In [ ]:
!tail Dados2.dat

Vamos usar a biblioteca do numpy para ler o arquivo:

In [14]:
np.loadtxt?

In [15]:
Dados= np.loadtxt('dados0.dat',skiprows=1) # Lê o arquivo de dados

In [16]:
np.shape(Dados) # Mostra a forma da matriz usada para armazenar os dados lidos

(5, 4)

In [18]:
Dados[:,0]

array([ 1.,  4.,  6., 12., 20.])

In [19]:
Dados

array([[ 1. ,  3. ,  0.5,  0.5],
       [ 4. , 19. ,  0.5,  0.5],
       [ 6. , 28. ,  0.5,  1.5],
       [12. , 44. ,  0.5,  2.5],
       [20. , 83. ,  0.5,  3.5]])

In [11]:
# Ajustando os dados para vetores
# Forma mais simples
x1=Dados[0:n-1,0]
y1=Dados[0:n-1,1]
y11=Dados[0:n-1,2]

NameError: name 'n' is not defined

In [ ]:
# Forma mais trabalhosa
for i in range(n-1):
    x[i] = Dados[i,0]          # cria um vetor de dados x-data
    y[i] = Dados[i,1]          # calcula o vetor de dados y-data
    y1[i] = Dados[i,2]

In [ ]:
# Fazendo um gráfico do resultado
largura = 10  # Largura da figura
altura  = 5   # Altura da figura
plt.figure(figsize =(largura, altura))     # Define o tamanho do gráfico
plt.grid()                       # Habilita a grade 
plt.xlabel('x')
plt.ylabel('y')
plt.title('Função transcendental')
plt.plot(x1, y1)   # Faz o gráfico
plt.plot(x1, y11)
plt.show()

## Algumas referências

A biblioteca [Matplotlib](http://matplotlib.org), possui uma vasta documentação além de muitos exemplos. Veja por exemplo:

- 1. Para técnicas de visualização de dados veja: http://matplotlib.org/users/recipes.html  
- 2. Para iniciar consulte o guia dos usuários: http://matplotlib.org/users/index.html